In [12]:
from pathos.multiprocessing import ProcessingPool as Pool
import bempp.api
import numpy as np
import os, time
from quadrature import *
from constants import *
from Grid_Maker import *
from Mesh_refine import *
import trimesh
from bempp.api.operators.potential import laplace as lp
from bempp.api.operators.boundary import sparse, laplace, modified_helmholtz
import pygamer
import threevis
from numba import jit

def U_tot_boundary(grid , q, x_q, return_time =False , save_plot=False , tolerance = 1e-8):
    '''
    Returns the total electrostatic potential in the boundary.
    Parameters:
    grid   : Bempp object
    '''
    init_time_spaces = time.time()
    potential.dirichl_space_u = bempp.api.function_space(grid,  mesh_info.u_space, mesh_info.u_order)
    potential.neumann_space_u = bempp.api.function_space(grid,  mesh_info.u_space, mesh_info.u_order) 
    potential.dual_to_dir_s_u = bempp.api.function_space(grid,  mesh_info.u_space, mesh_info.u_order)
    spaces_time      = time.time()-init_time_spaces
    U, dU , operators_time , assembly_time , GMRES_time , UdU_time , it_count= U_tot(potential.dirichl_space_u ,
                        potential.neumann_space_u , potential.dual_to_dir_s_u , tolerance, q, x_q)
    if save_plot:
        
        U_func  = bempp.api.GridFunction(potential.dirichl_space_u , fun=None, coefficients= U.coefficients.real)
        dU_func = bempp.api.GridFunction(potential.neumann_space_u , fun=None, coefficients=dU.coefficients.real)
        
        #bempp.api.export('Molecule/' + mesh_info.mol_name +'/' + mesh_info.mol_name + '_{0}{1}_U.vtk'.format(
                                      #mesh_info.mesh_density,   mesh_info.suffix )
                     #, grid_function = U_func , data_type = 'element')
        
        #bempp.api.export('Molecule/' + mesh_info.mol_name +'/' + mesh_info.mol_name + '_{0}{1}_dU.vtk'.format(
                                      #mesh_info.mesh_density,   mesh_info.suffix )
                     #, grid_function = dU_func , data_type = 'element')
    
    if return_time:
        return U, dU , spaces_time , operators_time , assembly_time , GMRES_time , UdU_time , it_count
    
    return U , dU

def U_tot(dirichl_space , neumann_space , dual_to_dir_s , tolerance, q, x_q):
    '''
    Computes the Total electrostatic mean potential on the boundary.
    Params:
    dirichl_space
    neumann_space
    dual_to_dir_s
    tolerance     : Tolerance of the solver
    '''
    
    init_time_operators = time.time()

    identity = sparse.identity(     dirichl_space, dirichl_space, dual_to_dir_s)
    slp_in   = laplace.single_layer(neumann_space, dirichl_space, dual_to_dir_s)
    dlp_in   = laplace.double_layer(dirichl_space, dirichl_space, dual_to_dir_s)

    slp_out  = modified_helmholtz.single_layer(neumann_space, dirichl_space, dual_to_dir_s, k)
    dlp_out  = modified_helmholtz.double_layer(dirichl_space, dirichl_space, dual_to_dir_s, k)
    #@bempp.api.real_callable
    #def q_times_G_L(x, n, domain_index, result):
    #    nrm = np.sqrt((x[0] - x_q[:, 0]) ** 2 + (x[1] - x_q[:, 1]) ** 2 + (x[2] - x_q[:, 2]) ** 2)
    #    aux = np.sum(q / nrm)
    #    result[0] = aux / (4 * np.pi * ep_m)
    
    @bempp.api.real_callable
    def q_times_G_L(x, n, domain_index, result):
        valor = np.empty (len(x_q))
        for i in range (len(x_q)):
            valor[i] = np.linalg.norm((x-x_q)[i])
        global ep_m
        result[:] = 1. / (4.*np.pi*ep_m)  * np.sum( q  / valor)
    #c = np.loadtxt ('coeficientesvicente')
        
    @bempp.api.real_callable    
    def zero_i(x, n, domain_index, result):
        result[:] = 0
    
    charged_grid_fun = bempp.api.GridFunction(dirichl_space, fun=q_times_G_L) #coefficients=c
    zero_grid_fun    = bempp.api.GridFunction(neumann_space, fun=zero_i     )
    #print (charged_grid_fun.coefficients.real)
    operators_time = time.time() - init_time_operators

    init_time_matrix = time.time()
    blocked = bempp.api.BlockedOperator(2, 2)
    blocked[0, 0] = 0.5*identity + dlp_in
    blocked[0, 1] = -slp_in
    blocked[1, 0] = 0.5*identity - dlp_out
    blocked[1, 1] = (ep_m/ep_s)*slp_out
    rhs = [charged_grid_fun, zero_grid_fun]
    
    assembly_time = time.time() - init_time_matrix
    
    
    
    print('GMRES Tolerance = {0}'.format(str(tolerance)))
    init_time_GMRES = time.time()
    sol, info,it_count = bempp.api.linalg.gmres( blocked, rhs, return_iteration_count=True , tol=tolerance ,
                                               restart = 300, use_strong_form=True)
    GMRES_time = time.time()-init_time_GMRES
    print("The linear system for U_tot was solved in {0} iterations".format(it_count))
    init_time_UdU = time.time()
    U , dU = sol
    UdU_time      = time.time()-init_time_UdU
    
           
    return U, dU , operators_time , assembly_time , GMRES_time , UdU_time , it_count

    
def S_trad_calc_R(dirichl_space, neumann_space, U, dU, x_q):

    # Se definen los operadores
    slp_in_O = lp.single_layer(neumann_space, x_q.transpose()) 
    dlp_in_O = lp.double_layer(dirichl_space, x_q.transpose())

    # Y con la solucion de las fronteras se fabrica el potencial evaluada en la posicion de cada carga
    U_R_O = slp_in_O * dU  -  dlp_in_O * U

    # Donde agregando algunas constantes podemos calcular la energia de solvatacion S
    
    S     = K * np.sum(mesh_info.q * U_R_O).real
    print("Solvation Energy : {:7.8f} [kcal/mol] ".format(S) )
    
    return S

def use_pygamer (face_array, vert_array, path, file_name, N_it_smoothing):
    '''
    Suaviza la malla generando ángulos de mínimo 15 y máximo 165.
    Evita triangulos alargados
    
    Parametros:
    face_array: archivo con las caras de la malla
    vert_array: archivo con los vértices de la malla
    path: lugar donde se encuentra
    file_name: nombre molecula
    N_it_smooothing: número de iteraciones para el smoothing de pygamer, cuidado, variar esto causa variaciones
    significativas. Generalmente uso 6, la idea es realizar los suficientes para no tener ningún ángulo fuera
    de los límites.
    
    Retorna:
    face_array y vert_array ya suavizados.
    
    '''
    #creo el archivo .off
    face_and_vert_to_off(face_array , vert_array , path , file_name)
    #mesh = pygamer.readOFF('Molecule/sphere_cent/sphere_cent_0-0.off')
    mesh = pygamer.readOFF(os.path.join( path , file_name + '.off' ))
    components, orientable, manifold = mesh.compute_orientation()
    mesh.correctNormals()
    # Set selection of all vertices to True so smooth will operate on them.
    for v in mesh.vertexIDs:
        v.data().selected = True
    # Apply N_it_smoothing iterations of smoothing
    mesh.smooth(max_iter=N_it_smoothing, preserve_ridges=True, verbose=True)
    print(F"The mesh currently has {mesh.nVertices} vertices, \
    {mesh.nEdges} edges, and {mesh.nFaces} faces.")
    
    pygamer.writeOFF(os.path.join(path, file_name + '.off'), mesh)
    
    new_off_file = open(os.path.join( path , file_name + '.off' ))
    algo = new_off_file.readlines()
    new_off_file.close()

    num_vert = int(algo[1].split()[0])
    num_face = int(algo[1].split()[1])

    vert_array = np.empty((0,3))
    for line in algo[2:num_vert+2]:
        vert_array = np.vstack((vert_array, line.split()))

    face_array = np.empty((0,3))
    for line in algo[num_vert+2:]:
        face_array = np.vstack((face_array, line.split()[1:]))

    vert_array = vert_array.astype(float)
    face_array = face_array.astype(int)+1
    
    return face_array, vert_array

def saved_sphere_distributions(name , r, q):
    '''
    Useful when running spherical grids.
    Inputs
    name  : Name of the distribution. Can be 'sphere_cent', 'sphere_offcent' or 'charge-dipole'.
    r     : Sphere radius
    Returns x_q , q 
    '''
        
    if name == 'sphere_cent':
        x_q = np.array( [[  1.E-12 ,  1.E-12 ,  1.E-12 ]]  )
        q = np.array( [q] )
    
    if name == 'sphere_offcent':
        x_q = np.array( [[  1.E-12 ,  1.E-12 ,   r/2. ]]  )
        q = np.array( [q] )
        
    if name == 'charge-dipole':
        x_q = np.array( [[  1.E-12 ,  1.E-12 ,  0.62 ],
                 [  1.E-12 ,  0.62*np.cos(np.pi*1.5 + 5.*np.pi/180.) ,
                                                      0.62*np.sin(np.pi*1.5 + 5.*np.pi/180. ) ] ,
                 [  1.E-12 ,  0.62*np.cos(np.pi*1.5 - 5.*np.pi/180.) ,
                                                      0.62*np.sin(np.pi*1.5 - 5.*np.pi/180. )  ]
                       ] )
        q = np.array( [1. , 1. , -1.]) 
    
    return x_q , q

def main ( name, dens, input_suffix, output_suffix, percentaje, N, N_ref, N_it_smoothing , smooth=True, Mallador='Nanoshaper', refine=True,  Use_GAMer = True,
          sphere=False, x_q = None, q = None, r = np.nan, fine_vert_array=None):
    
    if sphere:
        x_q, q = saved_sphere_distributions(name, r, q) 
        
        if x_q[0,0] == None or q[0] == None or r == np.nan:
            print('x_q, q or sphere radius where not defined.')
            
        mesh_info.mol_name     = name
        mesh_info.mesh_density = dens
        mesh_info.suffix       = input_suffix
        mesh_info.path         = os.path.join('Molecule' , mesh_info.mol_name)
            
        mesh_info.q , mesh_info.x_q = q , x_q
            
        init_time_0 = time.time()
        mesh_info.u_space , mesh_info.u_order     = 'DP' , 0
        mesh_info.phi_space , mesh_info.phi_order = 'P' , 1
        mesh_info.u_s_space , mesh_info.u_s_order = 'P' , 1            
            
        if input_suffix == '-0' and not sphere:
            grid = Grid_loader( mesh_info.mol_name , mesh_info.mesh_density , mesh_info.suffix , Mallador,  GAMer = Use_GAMer)
        else:
            #print('Loading previus mesh')
            #grid = Grid_loader( mesh_info.mol_name , mesh_info.mesh_density , mesh_info.suffix, Mallador)
                
            #para probar esferas bempp, r=radio, h=tamaño de los elementos: 
            grid = bempp.api.shapes.sphere(r = r, h = 0.1)
            face = np.transpose(grid.elements)
            vert = np.transpose(grid.vertices)
            vert_and_face_arrays_to_text_and_mesh( name , vert , face+1 ,
                                                input_suffix
                                                , dens=dens, Self_build=True)
            grid = Grid_loader( name, dens, input_suffix, 'Self')
        t_0 = time.time() - init_time_0
        print('Total time to create and load the mesh {0:.6f} [s]'.format(t_0))
            
        init_time = time.time()
        U, dU , spaces_time_U , operators_time_U , assembly_time_U , GMRES_time_U , UdU_time, it_count_U = U_tot_boundary(grid, mesh_info.q, mesh_info.x_q
                                                    , return_time = True , save_plot=True , tolerance = 1e-5) 
            
        S_trad = S_trad_calc_R( potential.dirichl_space_u, potential.neumann_space_u , U , dU, mesh_info.x_q )
        pot = potential_calc(grid, mesh_info.q , mesh_info.x_q)
        const_space = bempp.api.function_space(grid,  "DP", 0)   
        pot_F = bempp.api.GridFunction(const_space, fun=None, coefficients=np.abs(pot) )
        
        if True: #Marked elements      
            init_time_status = time.time()
            face_array = np.transpose(grid.elements) + 1    
            status = value_assignor_starter(face_array , np.abs(pot) , percentaje)
            const_space = bempp.api.function_space(grid,  "DP", 0)
            Status    = bempp.api.GridFunction(const_space, fun=None, coefficients=status)
            status_time = time.time()-init_time_status
            bempp.api.export('Molecule/' + name +'/' + name + '_{0}{1}_Marked_elements_{2}.msh'.format( 
                                            dens, input_suffix , N_ref )
                            , grid_function = Status , data_type = 'element')

        face_array = np.transpose(grid.elements)+1
        vert_array = np.transpose(grid.vertices)

        N_elements = len(face_array)
        smoothing_time = 0
        mesh_refiner_time = 0
        GAMer_time = 0
        if refine:
            init_time_mesh_refiner = time.time()
            new_face_array , new_vert_array = mesh_refiner(face_array , vert_array , np.abs(pot) , percentaje )
            vert_and_face_arrays_to_text_and_mesh( name , new_vert_array , new_face_array.astype(int)[:] ,
                                                    output_suffix, dens , Self_build=True)
            
            grid = Grid_loader( name , dens , output_suffix ,'Self')
            mesh_refiner_time  = time.time()-init_time_mesh_refiner              
            if smooth:
                init_time_smoothing = time.time()
                fine_vert_array = np.loadtxt('Molecule/{0}/{0}_40.0-0.vert'.format(mesh_info.mol_name))[:,:3]
                #fine_vert_array = text_to_list(name , '_40.0-0' , '.vert' , info_type=float)                
                aux_vert_array  = smoothing_vertex( new_vert_array , fine_vert_array, len(vert_array) )
                smoothing_time      = time.time()-init_time_smoothing
                #vert_and_face_arrays_to_text_and_mesh( name , aux_vert_array , new_face_array.astype(int)[:] ,
                #                                    output_suffix, dens , Self_build=True)
            elif not smooth:
                 aux_vert_array = new_vert_array.copy()

            
        if Use_GAMer:
            init_time_GAMer = time.time()
            new_face_array, new_vert_array = use_pygamer (new_face_array , aux_vert_array , mesh_info.path , 
                                                                mesh_info.mol_name+ '_' + str(dens) + output_suffix, N_it_smoothing)
            
            vert_and_face_arrays_to_text_and_mesh( name , new_vert_array , new_face_array.astype(int)[:] ,
                                                    output_suffix, dens , Self_build=True)
            
            grid = Grid_loader( name , dens , output_suffix ,'Self')
            GAMer_time = time.time()-init_time_GAMer

            
    else:
        mesh_info.mol_name     = name
        mesh_info.mesh_density = dens
        mesh_info.suffix       = input_suffix
        mesh_info.path         = os.path.join('Molecule' , mesh_info.mol_name)

        mesh_info.q , mesh_info.x_q = run_pqr(mesh_info.mol_name)
            
        init_time_0 = time.time()
        mesh_info.u_space , mesh_info.u_order     = 'DP' , 0
        mesh_info.phi_space , mesh_info.phi_order = 'P' , 1
        mesh_info.u_s_space , mesh_info.u_s_order = 'P' , 1
                
        if input_suffix == '-0' and not sphere:
            grid = Grid_loader( mesh_info.mol_name , mesh_info.mesh_density , mesh_info.suffix , Mallador, GAMer = False)
        else:
            print('Loading previus mesh')
            grid = Grid_loader( mesh_info.mol_name , mesh_info.mesh_density ,  mesh_info.suffix , 'Self')
        t_0 = time.time() - init_time_0
        print('Total time to create and load the mesh {0:.6f} [s]'.format(t_0))
    
        init_time_solving_out = time.time()
        U, dU , spaces_time_U , operators_time_U , assembly_time_U , GMRES_time_U , UdU_time, it_count_U = U_tot_boundary(grid, mesh_info.q, mesh_info.x_q
                                                , return_time = True , save_plot=True , tolerance = 1e-5)
    
        S_trad   = S_trad_calc_R( potential.dirichl_space_u, potential.neumann_space_u , U , dU, mesh_info.x_q )
        pot = potential_calc(grid, mesh_info.q , mesh_info.x_q)
        const_space = bempp.api.function_space(grid,  "DP", 0)   
        pot_F = bempp.api.GridFunction(const_space, fun=None, coefficients=np.abs(pot) )
    
        if True: #Marked elements      
            init_time_status = time.time()
            face_array = np.transpose(grid.elements) + 1    
            status = value_assignor_starter(face_array , np.abs(pot) , percentaje)
            const_space = bempp.api.function_space(grid,  "DP", 0)
            Status    = bempp.api.GridFunction(const_space, fun=None, coefficients=status)
            status_time = time.time()-init_time_status
            bempp.api.export('Molecule/' + name +'/' + name + '_{0}{1}_Marked_elements_{2}.msh'.format( 
                                            dens, input_suffix , N_ref )
                            , grid_function = Status , data_type = 'element')

        face_array = np.transpose(grid.elements)+1
        vert_array = np.transpose(grid.vertices)

        N_elements = len(face_array)
        smoothing_time = 0
        mesh_refiner_time = 0
        GAMer_time = 0
        if refine:
            init_time_mesh_refiner = time.time()
            new_face_array , new_vert_array = mesh_refiner(face_array , vert_array , np.abs(pot) , percentaje )
            vert_and_face_arrays_to_text_and_mesh( name , new_vert_array , new_face_array.astype(int)[:] ,
                                                    output_suffix, dens , Self_build=True)
            
            grid = Grid_loader( name , dens , output_suffix ,'Self')
            mesh_refiner_time  = time.time()-init_time_mesh_refiner              
            if smooth:
                init_time_smoothing = time.time()
                fine_vert_array = np.loadtxt('Molecule/{0}/{0}_40.0-0.vert'.format(mesh_info.mol_name))[:,:3]
                #fine_vert_array = text_to_list(name , '_40.0-0' , '.vert' , info_type=float)                
                aux_vert_array  = smoothing_vertex( new_vert_array , fine_vert_array, len(vert_array) )
                smoothing_time      = time.time()-init_time_smoothing
                #vert_and_face_arrays_to_text_and_mesh( name , aux_vert_array , new_face_array.astype(int)[:] ,
                #                                    output_suffix, dens , Self_build=True)
            elif not smooth:
                 aux_vert_array = new_vert_array.copy()

            
        if Use_GAMer:
            init_time_GAMer = time.time()
            new_face_array, new_vert_array = use_pygamer (new_face_array , aux_vert_array , mesh_info.path , 
                                                                mesh_info.mol_name+ '_' + str(dens) + output_suffix, N_it_smoothing)
            
            vert_and_face_arrays_to_text_and_mesh( name , new_vert_array , new_face_array.astype(int)[:] ,
                                                    output_suffix, dens , Self_build=True)
            
            grid = Grid_loader( name , dens , output_suffix ,'Self')
            GAMer_time = time.time()-init_time_GAMer



In [13]:
main('methanol', 0.5, '-0', '-1', 0.1, 25, 0, 6, smooth=True, Mallador = 'MSMS', refine =True, Use_GAMer = True,
                    sphere=False, x_q = None , 
                    q= None, r = np.nan , fine_vert_array = None)

Molecule/methanol/methanol_0.5-0.msh
.xyzr File from .pqr ready.
Normal .vert & .face Done
Loading the MSMS grid.
Molecule/methanol/methanol_0.5-0.msh
Normal .msh Done
Mesh Ready
Total time to create and load the mesh 0.066019 [s]
GMRES Tolerance = 1e-05


/home/elmobegazo/miniconda3/envs/bempp_prod_openmm/lib/python3.9/site-packages/scipy/sparse/linalg/dsolve/linsolve.py:318: SparseEfficiencyWarning: splu requires CSC matrix format
  warn('splu requires CSC matrix format', SparseEfficiencyWarning)


The linear system for U_tot was solved in 38 iterations
Solvation Energy : -5.17982076 [kcal/mol] 
Loading the built grid.
Molecule/methanol/methanol_0.5-1.msh
Molecule/methanol/methanol_0.5-1.msh
Initial Quality: Min Angle = 16.5007, Max Angle = 140.254, # smaller-than-15 = 0, # larger-than-165 = 0
Iteration 1:
Min Angle = 18.5411, Max Angle = 134.715, # smaller-than-15 = 0, # larger-than-165 = 0
Iteration 2:
Min Angle = 20.1963, Max Angle = 130.046, # smaller-than-15 = 0, # larger-than-165 = 0
Iteration 3:
Min Angle = 21.5447, Max Angle = 126.111, # smaller-than-15 = 0, # larger-than-165 = 0
Iteration 4:
Min Angle = 22.664, Max Angle = 122.758, # smaller-than-15 = 0, # larger-than-165 = 0
Iteration 5:
Min Angle = 23.6147, Max Angle = 119.861, # smaller-than-15 = 0, # larger-than-165 = 0
Iteration 6:
Min Angle = 24.4412, Max Angle = 117.323, # smaller-than-15 = 0, # larger-than-165 = 0
The mesh currently has 29 vertices,     81 edges, and 54 faces.
Loading the built grid.
Molecule/met